<a href="https://colab.research.google.com/github/muralidharan-1708/LLM-Projects/blob/main/Peft_and_Lora_finetune_falcon_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers datasets peft accelerate bitsandbytes
#transformers - load model
#datasets - to create and load dataset
#peft - for lora
#accelerate - optimize my training
# bitsandbytes - to initialize a same memory bits

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.0 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
# AutoModelForCausalLM – Loads a pre-trained model for causal language modeling
# AutoTokenizer – Loads the tokenizer associated with the model
# TrainingArguments – Configuration for training (e.g., epochs, batch size)
# Trainer – High-level API to train/evaluate models

from peft import get_peft_model, LoraConfig, TaskType
# get_peft_model – Applies PEFT (e.g., LoRA) to a base model
# LoraConfig – Configuration class for LoRA
# TaskType – Enum to specify task type (e.g., CAUSAL_LM)

from datasets import Dataset
# Dataset – Utility to create a Hugging Face dataset from Python data (e.g., dict, list)


In [3]:
data = {
    "text" : [
        "### Question : What is the capital of India ? ### Answer : New Delhi",
        "### Question : What is the national animal of India ? ### Answer : Royal Bengal Tiger",
        "### Question : What is the highest mountain peak in India ? ### Answer : Kanchenjunga"
    ]
}
dataset = Dataset.from_dict(data) #dict to dataset

In [4]:
model_name = "tiiuae/falcon-rw-1b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [5]:
# create a function for model-fitting
def tokenize(batch):
  tokens = tokenizer(
      batch["text"],
      padding = "max_length",
      truncation = True,  #truncate after crossing the max-length
      max_length = 128
  )
  #no label in text unlike image
  tokens['labels'] = tokens['input_ids'].copy()  #to remember the sequence Eg: I love cats. Love - I(label), cat - I love(label)
  return tokens

tokenized_dataset = dataset.map(tokenize, batched = True)

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [6]:
#initialize the model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit = True,
    device_map = "auto"  #for Hardware(between cpu and gpu)
)

config.json: 0.00B [00:00, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [7]:
#lora

lora_config = LoraConfig(
    r = 8, #r --> adapters
    lora_alpha = 16,  #scaling -->  limit or adjust the finetune of lora
    target_modules = ["query_key_value"],  # train only the attention mechanism (important words in a sentence val = 1)
    lora_dropout = 0.1,  #to stop the some nodes(when overfitting occurs )
    bias = "none",  #to find error
    task_type = TaskType.CAUSAL_LM  #peft tasktype, CAUSAL_LM - causal langauage model
)

In [8]:
# connecting the lora and model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 1,572,864 || all params: 1,313,198,080 || trainable%: 0.1198


In [9]:
#set the training parameters

training_args = TrainingArguments(
    output_dir = "./results",
    num_train_epochs = 3,   # number of complete process
    per_device_train_batch_size = 1,  #step by step
    learning_rate = 2e-4,
    logging_steps = 1,
    save_strategy = "no",
    report_to = "none"  #to get the alert or notifications
)

In [10]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_dataset,
    tokenizer = tokenizer
)

trainer.train()

/tmp/ipython-input-10-1711084281.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
1,5.881700
2,4.944600
3,4.204800
4,3.446200
5,2.984100
6,2.117500
7,1.861800
8,1.526900
9,1.440300


TrainOutput(global_step=9, training_loss=3.156447079446581, metrics={'train_runtime': 8.6372, 'train_samples_per_second': 1.042, 'train_steps_per_second': 1.042, 'total_flos': 8364732973056.0, 'train_loss': 3.156447079446581, 'epoch': 3.0})

In [11]:
model.save_pretrained("lora_fine-tuned-model")
tokenizer.save_pretrained("lora_fine-tuned-model")

('lora_fine-tuned-model/tokenizer_config.json',
 'lora_fine-tuned-model/special_tokens_map.json',
 'lora_fine-tuned-model/vocab.json',
 'lora_fine-tuned-model/merges.txt',
 'lora_fine-tuned-model/added_tokens.json',
 'lora_fine-tuned-model/tokenizer.json')